# Navier's solution to Beams and Plates

In [40]:
from dewloosh.solid.fourier import RectangularPlate, LoadGroup
from dewloosh.mesh import grid, PolyData
from dewloosh.mesh.topo.tr import Q4_to_T3
from dewloosh.mesh.space import StandardFrame
import numpy as np
from copy import deepcopy


In [41]:
Lx, Ly = 800., 400.
nx, ny = 5, 5
rx, ry = 10, 10
E = 2890.
nu = 0.2
t = 25.
x0, y0, w, h, q = 200., 100., 100., 200., -0.2


In [42]:
loads = {
    'LG1': {
        'LC1': {
            'type': 'rect',
            'points': [[0, 0], [10, 10]],
            'value': [0, 0, -10],
        },
        'LC2': {
            'type': 'rect',
            'region': [5., 6., 12., 10.],
            'value': [0, 0, -2],
        }
    },
    'LG2': {
        'LC3': {
            'type': 'point',
            'point': [10, 10],
            'value': [0, 0, -10],
        }
    },
    'dummy1': 10
}

LC = LoadGroup.from_dict(loads)


In [43]:
gridparams = {
    'size': (Lx, Ly),
    'shape': (rx, ry),
    'origo': (0, 0),
    'start': 0,
    'eshape': 'Q4'
}
coords_, topo = grid(**gridparams)
coords = np.zeros((coords_.shape[0], 3))
coords[:, :2] = coords_[:, :]
coords, triangles = Q4_to_T3(coords, topo)
GlobalFrame = StandardFrame(dim=3)
mesh = PolyData(coords=coords, topo=triangles, frame=GlobalFrame)


In [44]:
# material
G = E/2/(1+nu)
D = np.array([[1, nu, 0], [nu, 1, 0],
              [0., 0, (1-nu)/2]]) * t**3 * (E / (1-nu**2)) / 12
S = np.array([[G, 0], [0, G]]) * t * 5 / 6


In [45]:
P = RectangularPlate((Lx, Ly), (nx, ny), D=D, S=S)
P.add_loads_from_dict(deepcopy(loads))

LoadGroup({'LG1': LoadGroup({'LC1': RectLoad({'type': 'rect', 'key': 'LC1'}), 'LC2': RectLoad({'type': 'rect', 'key': 'LC2'}), 'key': 'LG1'}), 'LG2': LoadGroup({'LC3': PointLoad({'type': 'point', 'key': 'LC3'}), 'key': 'LG2'}), 'dummy1': 10})

In [46]:
P.model = 'm'
P.solve()
P.postproc(coords[:, :2], cleanup=False)

In [47]:
P.model = 'k'
P.solve()
P.postproc(coords[:, :2], cleanup=False)

In [48]:
for lc in P.loads.load_cases():
    print(lc.address)

['LG1', 'LC1']
['LG1', 'LC2']
['LG2', 'LC3']
